In [1]:
import numpy as np
import pandas as pd
import sys
import re
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [2]:
tweets = pd.read_csv('intellisent/tweets_new.csv')

tweets.dtypes.index
tweets = tweets[['text', 'tweet_class']]

smallest_len = min(len(tweets[tweets['tweet_class'] == 1]),
                len( tweets[tweets['tweet_class'] == -1]),
                len( tweets[tweets['tweet_class'] == 0]),
               len( tweets[tweets['tweet_class'] == 2]),
                  )

data = pd.DataFrame()

data = data.append(tweets[tweets['tweet_class']==-1])
data = data.append(tweets[tweets['tweet_class']==0][:smallest_len])
data = data.append(tweets[tweets['tweet_class']==1][:smallest_len])
data = data.append(tweets[tweets['tweet_class']==2][:smallest_len])

In [3]:
def clean(x):
    remove_url = re.sub(r'http\S+', '', x)
    remove_replies = re.sub(r'@\S+', '', remove_url)
    return remove_replies
    
data['cleaned'] = data['text'].apply(clean)

In [4]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['cleaned'].values)
X = tokenizer.texts_to_sequences(data['cleaned'].values)
X = pad_sequences(X)

In [5]:
model = Sequential()
model.add(Embedding(9000 + 1, 200, input_length = X.shape[1]))
model.add(Dropout(0.4))
model.add(LSTM(128))
model.add(Dense(64))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(4))
model.add(Activation('sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
data.loc[data.tweet_class == -1, 'tweet_class'] = 3

Y = data['tweet_class']

transformed_Y = to_categorical(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X,transformed_Y, test_size = 0.33, random_state = 42)

In [7]:
# checkpoint = ModelCheckpoint(filepath, monitor="loss", verbose=1, save_best_only=True, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.000001)
# print model.summary()
model.fit(X_train, Y_train, batch_size=1, epochs=7, validation_split=0.1, shuffle=True, callbacks=[reduce_lr])
# model.fit(X_train, Y_train, epochs = 7, batch_size=32, verbose = 2)

Train on 301 samples, validate on 34 samples
Epoch 1/7
301/301 [==============================] - 15s 50ms/step - loss: 1.4243 - acc: 0.2359 - val_loss: 1.3771 - val_acc: 0.2647
Epoch 2/7
301/301 [==============================] - 15s 49ms/step - loss: 1.3195 - acc: 0.4452 - val_loss: 1.2558 - val_acc: 0.4412
Epoch 3/7
301/301 [==============================] - 15s 50ms/step - loss: 0.9740 - acc: 0.6645 - val_loss: 1.2437 - val_acc: 0.5000
Epoch 4/7
301/301 [==============================] - 15s 51ms/step - loss: 0.5477 - acc: 0.8206 - val_loss: 1.3914 - val_acc: 0.5294
Epoch 5/7
301/301 [==============================] - 15s 51ms/step - loss: 0.2946 - acc: 0.9302 - val_loss: 1.4793 - val_acc: 0.5294
Epoch 6/7
301/301 [==============================] - 16s 52ms/step - loss: 0.1252 - acc: 0.9601 - val_loss: 1.5113 - val_acc: 0.5882
Epoch 7/7
301/301 [==============================] - 16s 52ms/step - loss: 0.1212 - acc: 0.9734 - val_loss: 1.9379 - val_acc: 0.4706


In [12]:
validation_size = 50

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = 128)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

ValueError: not enough values to unpack (expected 2, got 0)